In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator

In [2]:
# Load data
y = pd.read_csv("input/Ospiti.csv")
y["Data"] = pd.to_datetime(y["Data"]).dt.date
y 

,Data,Ospiti
0,2021-06-01,319.0
1,2021-06-02,396.0
2,2021-06-03,188.0
3,2021-06-04,258.0
4,2021-06-05,571.0
...,...,...
542,2022-11-26,732.0
543,2022-11-27,495.0
544,2022-11-28,226.0
545,2022-11-29,289.0


In [3]:
# normalize data with MinMaxScaler
scaler = MinMaxScaler()
y_scaled = scaler.fit_transform(y["Ospiti"].values.reshape(-1,1))
y_scaled = np.reshape(y_scaled, (1, len(y_scaled)))
y_scaled

array([[0.18949343, 0.26172608, 0.06660413, 0.13227017, 0.42589118,
        0.34803002, 0.09099437, 0.13320826, 0.24015009, 0.21294559,
        0.20262664, 0.33395872, 0.22045028, 0.09662289, 0.20731707,
        0.0750469 , 0.14446529, 0.21857411, 0.35928705, 0.21388368,
        0.09380863, 0.14446529, 0.20356473, 0.2054409 , 0.22045028,
        0.14821764, 0.19512195, 0.15572233, 0.18761726, 0.1641651 ,
        0.1575985 , 0.09474672, 0.34990619, 0.29268293, 0.17354597,
        0.07879925, 0.21763602, 0.26547842, 0.22326454, 0.33020638,
        0.06378987, 0.0891182 , 0.17260788, 0.16979362, 0.21669794,
        0.18667917, 0.43621013, 0.17729831, 0.19699812, 0.16510319,
        0.20450281, 0.20168856, 0.17917448, 0.33020638, 0.29831144,
        0.15478424, 0.17448405, 0.21482176, 0.18105066, 0.22795497,
        0.39118199, 0.24859287, 0.22326454, 0.18761726, 0.28611632,
        0.27861163, 0.16510319, 0.22514071, 0.14540338, 0.12382739,
        0.14071295, 0.12007505, 0.2532833 , 0.16

In [4]:
tf_dataset = tf.data.Dataset.from_tensor_slices(y_scaled[0])

2022-12-16 15:52:20.501645: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-16 15:52:20.501698: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (davide-Inspiron-3891): /proc/driver/nvidia/version does not exist
2022-12-16 15:52:20.502709: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
def make_window_dataset(ds, window_size=5, shift=1, stride=1):
  windows = ds.window(window_size, shift=shift, stride=stride)

  def sub_to_batch(sub):
    return sub.batch(window_size, drop_remainder=True)

  windows = windows.flat_map(sub_to_batch)
  return windows

In [6]:
ds = make_window_dataset(tf_dataset, window_size=37, shift = 1, stride=1)

for example in ds.take(10):
  print(example.numpy())

[0.18949343 0.26172608 0.06660413 0.13227017 0.42589118 0.34803002
 0.09099437 0.13320826 0.24015009 0.21294559 0.20262664 0.33395872
 0.22045028 0.09662289 0.20731707 0.0750469  0.14446529 0.21857411
 0.35928705 0.21388368 0.09380863 0.14446529 0.20356473 0.2054409
 0.22045028 0.14821764 0.19512195 0.15572233 0.18761726 0.1641651
 0.1575985  0.09474672 0.34990619 0.29268293 0.17354597 0.07879925
 0.21763602]
[0.26172608 0.06660413 0.13227017 0.42589118 0.34803002 0.09099437
 0.13320826 0.24015009 0.21294559 0.20262664 0.33395872 0.22045028
 0.09662289 0.20731707 0.0750469  0.14446529 0.21857411 0.35928705
 0.21388368 0.09380863 0.14446529 0.20356473 0.2054409  0.22045028
 0.14821764 0.19512195 0.15572233 0.18761726 0.1641651  0.1575985
 0.09474672 0.34990619 0.29268293 0.17354597 0.07879925 0.21763602
 0.26547842]
[0.06660413 0.13227017 0.42589118 0.34803002 0.09099437 0.13320826
 0.24015009 0.21294559 0.20262664 0.33395872 0.22045028 0.09662289
 0.20731707 0.0750469  0.14446529 0.218

In [7]:
def dense_7_step(batch):
  # Shift features and labels one step relative to each other.
  return batch[:-7], batch[-7:]

In [8]:
dense_labels_ds = ds.map(dense_7_step)

#create a tensor for holding the labels and features
all_inputs = []
all_labels = []

for inputs,labels in dense_labels_ds.take(3):
  print(inputs.numpy(), "=>", labels.numpy())

for inputs,labels in dense_labels_ds:
  all_inputs.append(inputs)
  all_labels.append(labels)

[0.18949343 0.26172608 0.06660413 0.13227017 0.42589118 0.34803002
 0.09099437 0.13320826 0.24015009 0.21294559 0.20262664 0.33395872
 0.22045028 0.09662289 0.20731707 0.0750469  0.14446529 0.21857411
 0.35928705 0.21388368 0.09380863 0.14446529 0.20356473 0.2054409
 0.22045028 0.14821764 0.19512195 0.15572233 0.18761726 0.1641651 ] => [0.1575985  0.09474672 0.34990619 0.29268293 0.17354597 0.07879925
 0.21763602]
[0.26172608 0.06660413 0.13227017 0.42589118 0.34803002 0.09099437
 0.13320826 0.24015009 0.21294559 0.20262664 0.33395872 0.22045028
 0.09662289 0.20731707 0.0750469  0.14446529 0.21857411 0.35928705
 0.21388368 0.09380863 0.14446529 0.20356473 0.2054409  0.22045028
 0.14821764 0.19512195 0.15572233 0.18761726 0.1641651  0.1575985 ] => [0.09474672 0.34990619 0.29268293 0.17354597 0.07879925 0.21763602
 0.26547842]
[0.06660413 0.13227017 0.42589118 0.34803002 0.09099437 0.13320826
 0.24015009 0.21294559 0.20262664 0.33395872 0.22045028 0.09662289
 0.20731707 0.0750469  0.1444

In [9]:
X = tf.concat(all_inputs, axis=0)
X = tf.reshape(X, (-1, 30))
X

<tf.Tensor: shape=(511, 30), dtype=float64, numpy=
array([[0.18949343, 0.26172608, 0.06660413, ..., 0.15572233, 0.18761726,
        0.1641651 ],
       [0.26172608, 0.06660413, 0.13227017, ..., 0.18761726, 0.1641651 ,
        0.1575985 ],
       [0.06660413, 0.13227017, 0.42589118, ..., 0.1641651 , 0.1575985 ,
        0.09474672],
       ...,
       [0.43621013, 0.13320826, 0.1782364 , ..., 0.54409006, 0.39868668,
        0.12382739],
       [0.13320826, 0.1782364 , 0.17917448, ..., 0.39868668, 0.12382739,
        0.19418386],
       [0.1782364 , 0.17917448, 0.28893058, ..., 0.12382739, 0.19418386,
        0.18574109]])>

In [10]:
Y = tf.concat(all_labels, axis=0)
Y = tf.reshape(Y, (-1, 7))
Y

<tf.Tensor: shape=(511, 7), dtype=float64, numpy=
array([[0.1575985 , 0.09474672, 0.34990619, ..., 0.17354597, 0.07879925,
        0.21763602],
       [0.09474672, 0.34990619, 0.29268293, ..., 0.07879925, 0.21763602,
        0.26547842],
       [0.34990619, 0.29268293, 0.17354597, ..., 0.21763602, 0.26547842,
        0.22326454],
       ...,
       [0.19418386, 0.18574109, 0.18574109, ..., 0.57692308, 0.35459662,
        0.10225141],
       [0.18574109, 0.18574109, 0.24202627, ..., 0.35459662, 0.10225141,
        0.16135084],
       [0.18574109, 0.24202627, 0.57692308, ..., 0.10225141, 0.16135084,
        0.12195122]])>

### Build a simple dense model

In [11]:
# build a forecasting model using the Keras Sequential API
model = keras.Sequential(
    [
        layers.Dense(256, activation="relu", name="layer1"),
        # add dropout to prevent overfitting
        layers.Dropout(0.2),
        layers.Dense(128, activation="relu", name="layer2"),
        layers.Dropout(0.2),
        layers.Dense(64, activation="relu", name="layer3"),
        layers.Dropout(0.2),
        layers.Dense(7, name="layer4"),
    ]
)

In [12]:
model.compile(optimizer='adam', loss="mse")

In [13]:
model.fit(x=X, y=Y ,epochs=500)

Epoch 1/500
16/16 [==============================] - 1s 2ms/step - loss: 0.0470
Epoch 2/500
16/16 [==============================] - 0s 1ms/step - loss: 0.0218
Epoch 3/500
16/16 [==============================] - 0s 1ms/step - loss: 0.0171
Epoch 4/500
16/16 [==============================] - 0s 2ms/step - loss: 0.0151
Epoch 5/500
16/16 [==============================] - 0s 2ms/step - loss: 0.0138
Epoch 6/500
16/16 [==============================] - 0s 2ms/step - loss: 0.0132
Epoch 7/500
16/16 [==============================] - 0s 2ms/step - loss: 0.0126
Epoch 8/500
16/16 [==============================] - 0s 2ms/step - loss: 0.0113
Epoch 9/500
16/16 [==============================] - 0s 3ms/step - loss: 0.0114
Epoch 10/500
16/16 [==============================] - 0s 3ms/step - loss: 0.0107
Epoch 11/500
16/16 [==============================] - 0s 3ms/step - loss: 0.0105
Epoch 12/500
16/16 [==============================] - 0s 2ms/step - loss: 0.0102
Epoch 13/500
16/16 [=================

### Simulate a request for forecasting the next 7 days

In [14]:
y_test = y.iloc[-30:]
y_test

,Data,Ospiti
517,2022-11-01,489.0
518,2022-11-02,221.0
519,2022-11-03,285.0
520,2022-11-04,371.0
521,2022-11-05,701.0
522,2022-11-06,435.0
523,2022-11-07,206.0
524,2022-11-08,294.0
525,2022-11-09,349.0
526,2022-11-10,304.0


In [15]:
y_test_scaled = scaler.fit_transform(y_test["Ospiti"].values.reshape(-1,1))
y_test_scaled = np.reshape(y_test_scaled, (1, len(y_test_scaled)))
y_test_scaled

array([[0.47580645, 0.04354839, 0.14677419, 0.28548387, 0.81774194,
        0.38870968, 0.01935484, 0.16129032, 0.25      , 0.17741935,
        0.36290323, 1.        , 0.38225806, 0.        , 0.14516129,
        0.20645161, 0.18064516, 0.44354839, 0.81129032, 0.56129032,
        0.08870968, 0.20967742, 0.19516129, 0.19516129, 0.29193548,
        0.86774194, 0.48548387, 0.0516129 , 0.15322581, 0.08548387]])

In [16]:
# dates to be predicted
dates = pd.date_range(start=y["Data"].iloc[-1],periods=8)[1:]
dates

DatetimeIndex(['2022-12-01', '2022-12-02', '2022-12-03', '2022-12-04',
               '2022-12-05', '2022-12-06', '2022-12-07'],
              dtype='datetime64[ns]', freq='D')

In [17]:
# predict the next 7 days
pred = model.predict(y_test_scaled)
pred

array([[0.19687265, 0.31422597, 0.7861344 , 0.40817857, 0.10498112,
        0.17009383, 0.20105983]], dtype=float32)

In [18]:
pred_unscaled = scaler.inverse_transform(pred)
pred_unscaled

array([[316.06104, 388.8201 , 681.4033 , 447.0707 , 259.0883 , 299.4582 ,
        318.65707]], dtype=float32)

In [19]:
# construct results dataframe
results = pd.DataFrame(pred_unscaled[0], columns=["Ospiti"], index=dates)
results

,Ospiti
2022-12-01,316.061035
2022-12-02,388.820099
2022-12-03,681.403320
2022-12-04,447.070709
2022-12-05,259.088287
2022-12-06,299.458191
2022-12-07,318.657074


In [20]:
# save the model with pickle
import pickle
pickle.dump(model, open("models/my_dense_model", 'wb'))

# save the scaler with pickle
pickle.dump(scaler, open("models/scaler", 'wb'))


2022-12-16 15:52:44.290360: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ram://c7637a5d-34e6-4c3d-9968-23def59bfaf5/assets
